# Object Detection Example

## Introduction

In this notebook, we'll walk-through a detailed example of how you can use Velour to evaluate object detections made on [the COCO Panoptic dataset](https://cocodataset.org/#home). We'll use Ultralytics' `YOLOv8` model to predict what objects exist in various COCO photographs.

For a conceptual introduction to Velour, [check out our project overview](https://striveworks.github.io/velour/). For a higher-level example notebook, [check out our "Getting Started" notebook](https://github.com/Striveworks/velour/blob/main/examples/getting_started.ipynb).

## Defining Our Datasets

We start by fetching our dataset and uploading it to Velour.

In [ ]:
from tqdm import tqdm
from pathlib import Path
from velour import Client, Model, Annotation, Prediction, Label
from velour.enums import TaskType
from velour.viz import create_combined_segmentation_mask

import integrations.coco_integration as coco
import integrations.yolo_integration as yolo

import ultralytics

# connect to Velour API
client = Client("http://localhost:8000")

In [ ]:
# create the dataset in Velour; see the scripts in `integrations/` for code
velour_dataset = coco.create_dataset_from_coco_panoptic(
    client, 
    destination=Path("./").absolute().parent / Path("coco"),
    limit=2, 
)

## Defining Our Model

With our `Dataset` in Velour, we're ready to create our `Model` object and add `Predictions` to it.

In [ ]:
# define the model in Velour. note that we can use any name we'd like
velour_model = Model(client, "yolov8n-seg")

inference_engine = ultralytics.YOLO(f"{velour_model.name}.pt")

for datum in tqdm(velour_dataset.get_datums()):

    image = coco.download_image(datum)

    results = inference_engine(image, verbose=False)

    # convert result into Velour Bounding Box prediction
    bbox_prediction = yolo.parse_detection_into_bounding_box(
        results,            # raw inference
        datum=datum,        # velour datum
        label_key='name',   # label_key override
    )

    # convert result into Velour Raster prediction
    raster_prediction = yolo.parse_detection_into_raster(
        results,            # raw inference
        datum=datum,        # velour datum
        label_key='name',   # label_key override
    )

    # add predictions to the model
    velour_model.add_prediction(bbox_prediction)
    velour_model.add_prediction(raster_prediction)

Lastly, we finalize our `Model` to get it ready for evaluation.

In [ ]:
velour_model.finalize_inferences(velour_dataset)

## Exploring Our Dataset

Before we evaluate our results, let's check out the metadata stored in Velour. Below, we show an example of a COCO image (in this case, the image we added using UID '139').

In [ ]:
groundtruth_139 = velour_dataset.get_groundtruth('139')
coco.download_image(groundtruth_139.datum)

Next, we overlay a segmentation mask over the image to show all of the objects we want to be able to detect.

In [ ]:
instance_mask, instance_legend = create_combined_segmentation_mask(
    [groundtruth_139], 
    label_key="name",
    task_type=TaskType.DETECTION,
)

instance_mask

In [ ]:
# print the color code for the above segmentations
for k, v in instance_legend.items():
    print(k)
    display(v)

## Evaluating Performance

With our `Dataset` and `Model` defined, we're ready to evaluate our performance and display the results. Note that we use the `wait_for_completion` method since all evaluations run as a postgres `BackgroundTask`; this method ensures that the evaluation finishes before we display the results.

In [ ]:
eval1 = velour_model.evaluate_detection(velour_dataset)
eval1.wait_for_completion()

eval1.results

### Evaluating with Filters

Sometimes, we may only want to calculate metrics for a subset of our data (i.e., we may only want to see how well our model performed at a specific type of detection). To accomplish this task, we can use the `filters` param of `evaluation_detection` to specify what types of data to evaluate performance for.

In [ ]:
# see how well we did at detecting people in images that have segmentation areas <=2000 pixels
eval2 = velour_model.evaluate_detection(
    velour_dataset,
    filters=[
        Label.label == Label(key='name', value='person'),
        Annotation.geometric_area <= 2000
    ]
)
eval2.wait_for_completion()
eval2.results